In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import numpy as np


In [2]:
csv_file = os.path.join( "Resources", "president_polls.csv")
polls_data_df = pd.read_csv(csv_file)
polls_data_df.head()

,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,...,created_at,notes,url,stage,race_id,answer,candidate_id,candidate_name,candidate_party,pct
0,134285,71678,2020,Pennsylvania,383,Public Policy Polling,1484,American Bridge 21st Century,Public Policy Polling,263.0,...,10/22/2020 15:33,NaN,https://americanbridgepac.org/wp-content/uploa...,general,6249,Biden,13256,Joseph R. Biden Jr.,DEM,51.0
1,134285,71678,2020,Pennsylvania,383,Public Policy Polling,1484,American Bridge 21st Century,Public Policy Polling,263.0,...,10/22/2020 15:33,NaN,https://americanbridgepac.org/wp-content/uploa...,general,6249,Trump,13254,Donald Trump,REP,46.0
2,134286,71679,2020,Michigan,383,Public Policy Polling,1484,American Bridge 21st Century,Public Policy Polling,263.0,...,10/22/2020 15:33,NaN,https://americanbridgepac.org/wp-content/uploa...,general,6233,Biden,13256,Joseph R. Biden Jr.,DEM,50.0
3,134286,71679,2020,Michigan,383,Public Policy Polling,1484,American Bridge 21st Century,Public Policy Polling,263.0,...,10/22/2020 15:33,NaN,https://americanbridgepac.org/wp-content/uploa...,general,6233,Trump,13254,Donald Trump,REP,43.0
4,134249,71662,2020,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,NaN,NaN,Rasmussen Reports/Pulse Opinion Research,277.0,...,10/22/2020 10:27,NaN,https://www.rasmussenreports.com/public_conten...,general,6238,Biden,13256,Joseph R. Biden Jr.,DEM,48.0


In [3]:
new_polls_data_df = polls_data_df[[ 'state', 'pollster_id', 'pollster', 'fte_grade', 'sample_size', 'population', 'methodology', 'end_date', 'nationwide_batch', 'answer', 'candidate_party', 'pct' ]].copy()
new_polls_data_df.head()

,state,pollster_id,pollster,fte_grade,sample_size,population,methodology,end_date,nationwide_batch,answer,candidate_party,pct
0,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Biden,DEM,51.0
1,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Trump,REP,46.0
2,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Biden,DEM,50.0
3,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Trump,REP,43.0
4,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,C+,800.0,lv,IVR/Online,10/21/2020,False,Biden,DEM,48.0


In [4]:
only_trump_and_biden = new_polls_data_df[(new_polls_data_df["answer"] == "Trump") | (new_polls_data_df["answer"] == "Biden")]
only_trump_and_biden["answer"].value_counts()


Trump    5939
Biden    4426
Name: answer, dtype: int64

# Which colums have null data?

In [5]:
only_trump_and_biden.isnull().any()

state                True
pollster_id         False
pollster            False
fte_grade            True
sample_size          True
population          False
methodology          True
end_date            False
nationwide_batch    False
answer              False
candidate_party     False
pct                 False
dtype: bool

**Columns with null values: state, fte_grade, sample_size, and methodology**

In [6]:
only_trump_and_biden.count()

state                6761
pollster_id         10365
pollster            10365
fte_grade            9716
sample_size         10363
population          10365
methodology         10341
end_date            10365
nationwide_batch    10365
answer              10365
candidate_party     10365
pct                 10365
dtype: int64

In [7]:
only_trump_and_biden.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
12804    False
12805    False
12807    False
12809    False
12811     True
Length: 10365, dtype: bool

In [8]:
only_trump_and_biden[only_trump_and_biden.isnull().any(axis=1)]

,state,pollster_id,pollster,fte_grade,sample_size,population,methodology,end_date,nationwide_batch,answer,candidate_party,pct
8,NaN,1193,SurveyMonkey,D-,13943.0,rv,Online,10/21/2020,False,Biden,DEM,52.0
9,NaN,1193,SurveyMonkey,D-,13943.0,rv,Online,10/21/2020,False,Trump,REP,45.0
10,NaN,1193,SurveyMonkey,D-,13943.0,lv,Online,10/21/2020,False,Biden,DEM,52.0
11,NaN,1193,SurveyMonkey,D-,13943.0,lv,Online,10/21/2020,False,Trump,REP,46.0
12,NaN,228,IBD/TIPP,A/B,965.0,lv,Live Phone/Online,10/21/2020,False,Biden,DEM,49.6
...,...,...,...,...,...,...,...,...,...,...,...,...
12796,NaN,1416,Harris Insights & Analytics,C,1001.0,rv,Online,12/17/2018,False,Biden,DEM,42.0
12797,NaN,1416,Harris Insights & Analytics,C,1001.0,rv,Online,12/17/2018,False,Trump,REP,36.0
12799,NaN,1416,Harris Insights & Analytics,C,1001.0,rv,Online,12/17/2018,False,Trump,REP,37.0
12801,NaN,1416,Harris Insights & Analytics,C,1001.0,rv,Online,12/17/2018,False,Trump,REP,37.0


**There are 4,005 rows out of 10,365 rows with null data. Since we are interested in state polling data in relations to our financial data by state, we will drop all rows that contain null values in state column.**

In [9]:
only_trump_and_biden = only_trump_and_biden.dropna(axis=0 , how='any', subset = ['state'])

In [10]:
only_trump_and_biden.isnull().any()

state               False
pollster_id         False
pollster            False
fte_grade            True
sample_size         False
population          False
methodology          True
end_date            False
nationwide_batch    False
answer              False
candidate_party     False
pct                 False
dtype: bool

In [11]:
only_trump_and_biden.count()

state               6761
pollster_id         6761
pollster            6761
fte_grade           6379
sample_size         6761
population          6761
methodology         6740
end_date            6761
nationwide_batch    6761
answer              6761
candidate_party     6761
pct                 6761
dtype: int64

**Above, we see now that the State column is "False" for containing any nulls. We now have 6,761 rows with State data for the candidates (Trump/Biden)**

# Add a new column called committee_id for Trump and Biden

In [12]:
only_trump_and_biden['committee_id']=''
only_trump_and_biden.head()

,state,pollster_id,pollster,fte_grade,sample_size,population,methodology,end_date,nationwide_batch,answer,candidate_party,pct,committee_id
0,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Biden,DEM,51.0,
1,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Trump,REP,46.0,
2,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Biden,DEM,50.0,
3,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Trump,REP,43.0,
4,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,C+,800.0,lv,IVR/Online,10/21/2020,False,Biden,DEM,48.0,


In [13]:
only_trump_and_biden.loc[only_trump_and_biden['answer'] == 'Trump', 'committee_id'] = "C00580100"
only_trump_and_biden.loc[only_trump_and_biden['answer'] == 'Biden', 'committee_id'] = "C00703975"
only_trump_and_biden.head()

,state,pollster_id,pollster,fte_grade,sample_size,population,methodology,end_date,nationwide_batch,answer,candidate_party,pct,committee_id
0,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Biden,DEM,51.0,C00703975
1,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,10/22/2020,False,Trump,REP,46.0,C00580100
2,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Biden,DEM,50.0,C00703975
3,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,10/22/2020,False,Trump,REP,43.0,C00580100
4,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,C+,800.0,lv,IVR/Online,10/21/2020,False,Biden,DEM,48.0,C00703975


# Evaluate what type of values for states are present?

In [14]:
only_trump_and_biden['state'].value_counts()

Wisconsin         672
Michigan          390
Pennsylvania      386
North Carolina    369
Florida           322
Arizona           306
Texas             302
Georgia           203
Iowa              179
California        176
Maine             173
Ohio              163
New Hampshire     146
Virginia          135
Minnesota         130
South Carolina    119
Nevada            113
Colorado          113
Montana            99
Missouri           96
Utah               95
New Jersey         94
Kentucky           89
Kansas             84
Washington         84
North Dakota       81
New York           79
Alaska             78
Alabama            77
Connecticut        77
Maryland           75
Oklahoma           73
Massachusetts      72
New Mexico         71
Nebraska           70
Mississippi        67
Tennessee          66
Delaware           66
Indiana            66
Louisiana          64
West Virginia      63
Arkansas           60
Oregon             60
Hawaii             60
Vermont            58
Illinois  

In [15]:
output_file = os.path.join("Output", "polls_clean_df.csv")

only_trump_and_biden.to_csv(output_file, index=False, header=True)

# Connect to local database

In [20]:
pg_user = 'postgres'
pg_password = 'postgres'
db_name = 'POTUS_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
engine.table_names()

['States', 'Candidates', 'Candidate_Spending', 'Polls']

In [22]:
only_trump_and_biden.to_sql(name="Polls", con=engine, if_exists='append', index=False)

In [23]:
pd.read_sql_query('select * from "Polls"', con=engine).head(50)

,state,pollster_id,pollster,fte_grade,sample_size,population,methodology,end_date,nationwide_batch,answer,candidate_party,pct,committee_id,id
0,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,2020-10-22,False,Biden,DEM,51.0,C00703975,1
1,Pennsylvania,383,Public Policy Polling,B,890.0,v,IVR/Text,2020-10-22,False,Trump,REP,46.0,C00580100,2
2,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,2020-10-22,False,Biden,DEM,50.0,C00703975,3
3,Michigan,383,Public Policy Polling,B,804.0,v,IVR/Text,2020-10-22,False,Trump,REP,43.0,C00580100,4
4,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,C+,800.0,lv,IVR/Online,2020-10-21,False,Biden,DEM,48.0,C00703975,5
5,North Carolina,399,Rasmussen Reports/Pulse Opinion Research,C+,800.0,lv,IVR/Online,2020-10-21,False,Trump,REP,49.0,C00580100,6
6,Iowa,1102,Emerson College,A-,435.0,lv,IVR/Online/Text,2020-10-21,False,Biden,DEM,48.1,C00703975,7
7,Iowa,1102,Emerson College,A-,435.0,lv,IVR/Online/Text,2020-10-21,False,Trump,REP,48.0,C00580100,8
8,Pennsylvania,1447,Civiqs,B/C,1577.0,a,Online,2020-10-21,False,Biden,DEM,52.0,C00703975,9
9,Pennsylvania,1447,Civiqs,B/C,1577.0,a,Online,2020-10-21,False,Trump,REP,46.0,C00580100,10
